In [94]:
!pip install newspaper3k
!pip install --upgrade lxml newspaper3k lxml_html_clean
!pip install fuzzywuzzy

### 코드

In [97]:
import urllib.request
import datetime
import json
import pandas as pd
from urllib.parse import quote

# ✅ 네이버 API 인증 정보
client_id = "IamYBKISQmZU5MdHRnsm"
client_secret = "AEZKqyZhux"

# ✅ 검색할 주제와 관련 키워드 리스트
related_keywords = {
    '정보보안': ['사이버', '해킹', '랜섬웨어', '보안', '위협', '데이터', '보호', '보안 사고', '시스템', '네트워크', '개인정보', '정보보안'],
    '클라우드': ['컴퓨팅', '서버', '데이터', '가상화', '서비스', '온라인', '인터넷', 'IT',  '인프라', '기업', '웹'],
    '제로 트러스트': ['보안', '정책', '접근', '제어', '네트워크', '보안', '인증', '데이터', '기업', '인증', '비밀번호', '권한', '관리'],
    '정보유출사고': ['데이터 유출', '보안 사고', '개인정보', '기업 보안', '고객 정보', '정보 보호', '정보 유출', '사이버 공격', '데이터 침해'],
    '나이스': ['교육', '시스템', '학교', '관리', '행정', '학생', '정보', '교사', '업무', '교육청', '전자', '문서', '공공', 'IT', '디지털', '행정', 'NEIS'],
    '블록체인': ['디지털', '자산', '거래', '시스템', '보안', '기술', '데이터', '무결성', '탈중앙화', '네트워크', '금융', '기술', '온라인', '거래', '암호화', '블록체인'],
    'STO': ['증권', '투자', '자산', '관리', '금융', '시장', '거래', '블록체인', '금융', '기업', '투자', '디지털', '금융', '경제', 'STO'],
    'AI': ['인공지능', '기술', '데이터', '분석', '컴퓨터', '자동화', '빅데이터', '딥러닝', '소프트웨어', '산업', '혁신', 'AI'],
    '망분리': ['보안', '네트워크', '기업 보안', '인터넷', '분리', '데이터', '보호', '네트워크', '관리', '정보', '보호', '보안', '인프라', '기업', 'IT', '망분리'],
    'IT업계': ['소프트웨어', '하드웨어', '기업', '스타트업', '기술 산업', '디지털', '혁신', '시장', '비즈니스', 'IT'],
    '보안업계': ['보안', '기술', '보안', '서비스', '위협', '관리', '사이버', '공격', '정보', '보호', '보안', '솔루션', '기업', '보안', '데이터', '관리']
}

# ✅ 제외할 키워드 리스트 (불필요한 뉴스 필터링)
EXCLUDE_KEYWORDS = ["연예", "스포츠", "날씨", "범죄", "게임", "드라마", "영화", "NBA", "축제", "추위", "호텔", "가수", "아이돌"]

# ✅ 네이버 API 요청 함수
def get_request_url(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# ✅ 네이버 뉴스 검색 API 호출 함수 (200개 가져오기)
def get_naver_search(keyword):
    base_url = "https://openapi.naver.com/v1/search/news.json"
    all_items = []  # 모든 결과 저장

    # 100개씩 2번 요청 (총 200개 확보)
    for start in [1, 101]:
        url = f"{base_url}?query={quote(keyword)}&start={start}&display=100&sort=date"  # 최신순 정렬
        response = get_request_url(url)

        if response:
            data = json.loads(response)
            all_items.extend(data.get('items', []))  # 결과 리스트 합침

    return all_items  # 총 200개 뉴스 반환

# ✅ 뉴스 데이터 필터링 함수
def filter_news(post, keywords):
    """
    기사 제목이나 설명에 주어진 키워드 리스트의 단어가 포함되고,
    EXCLUDE_KEYWORDS(제외할 키워드)가 포함되지 않은 경우만 True 반환
    """
    title = post['title']
    description = post['description']

    # 포함할 키워드 체크 (하나라도 포함되면 OK)
    if not any(keyword in title or keyword in description for keyword in keywords):
        return False  # 관련 키워드가 포함되지 않은 기사

    # 제외할 키워드 체크 (하나라도 포함되면 False)
    if any(keyword in title or keyword in description for keyword in EXCLUDE_KEYWORDS):
        return False  # 제외할 키워드가 포함된 기사이므로 필터링

    return True  # 관련 키워드가 포함되며, 제외할 키워드가 포함되지 않은 경우

# ✅ 검색 실행 함수
def main():

    all_news = []  # 모든 주제의 뉴스를 저장할 리스트
    total_cnt = 0  # 전체 뉴스 개수 카운트

    for topic, keywords in related_keywords.items():
        json_result = []
        cnt = 0

        # ✅ API 요청 (200개 가져오기)
        json_response = get_naver_search(topic)

        if not json_response:
            print(f"❌ '{topic}'에 대한 뉴스 검색 결과가 없습니다.")
            continue

        print(f"🔎 '{topic}' 검색 결과: {len(json_response)}건 (최대 200개 검색)")

        # ✅ 뉴스 가져오기 및 날짜 + 키워드 필터링
        for post in json_response:
            # 날짜 변환 (YYYYMMDD)
            p_date = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
            p_date = p_date.strftime('%Y%m%d')

            # ✅ 입력한 날짜와 일치하고, 관련 키워드 포함 & 불필요한 뉴스 제외
            if p_date == target_date and filter_news(post, keywords):
                cnt += 1
                json_result.append({
                    'cnt': cnt,
                    'topic': topic,  # 주제명 추가
                    'title': post['title'],
                    'description': post['description'],
                    'org_link': post['originallink'],
                    'link': post['link'],
                    'pDate': p_date
                })

        # ✅ 필터링된 뉴스만 리스트에 추가
        all_news.extend(json_result)
        total_cnt += len(json_result)

    # ✅ JSON 저장
    filename = f"Filtered_News_{target_date}.json"
    with open(filename, 'w', encoding='utf8') as outfile:
        json.dump(all_news, outfile, indent=4, sort_keys=True, ensure_ascii=False)

    print(f"✅ 총 가져온 데이터: {total_cnt}건")
    print(f"📂 저장 완료: {filename}")

    # ✅ 결과를 DataFrame으로 변환하여 CSV 저장
    df = pd.DataFrame(all_news)
    if not df.empty:
        df.to_csv(f"Filtered_News_{target_date}.csv", encoding='utf-8-sig', index=False)
        print(f"📂 CSV 저장 완료: Filtered_News_{target_date}.csv")
        display(df)
    else:
        print("❌ 해당 날짜에 대한 관련 뉴스가 없습니다.")

if __name__ == '__main__':
    main()


📅 검색할 날짜를 입력하세요 (YYYYMMDD): 20250205
🔎 '정보보안' 검색 결과: 200건 (최대 200개 검색)
🔎 '클라우드' 검색 결과: 200건 (최대 200개 검색)
🔎 '제로 트러스트' 검색 결과: 200건 (최대 200개 검색)
🔎 '정보유출사고' 검색 결과: 200건 (최대 200개 검색)
🔎 '나이스' 검색 결과: 200건 (최대 200개 검색)
🔎 '블록체인' 검색 결과: 200건 (최대 200개 검색)
🔎 'STO' 검색 결과: 200건 (최대 200개 검색)
🔎 'AI' 검색 결과: 200건 (최대 200개 검색)
🔎 '망분리' 검색 결과: 200건 (최대 200개 검색)
🔎 'IT업계' 검색 결과: 200건 (최대 200개 검색)
🔎 '보안업계' 검색 결과: 200건 (최대 200개 검색)
✅ 총 가져온 데이터: 674건
📂 저장 완료: Filtered_News_20250205.json
📂 CSV 저장 완료: Filtered_News_20250205.csv


,cnt,topic,title,description,org_link,link,pDate
0,1,정보보안,"슈프리마, '바이오스타 2 VMS' GS인증 1등급 획득","한국인터넷진흥원, 한국정보보호산업협회 -일시: 2025년 2월 11일 화요일 오전 ...",https://www.dailysecu.com/news/articleView.htm...,https://www.dailysecu.com/news/articleView.htm...,20250205
1,2,정보보안,"아이즈비전, ‘쯔양 요금제’ 출시…“사회적 가치ㆍ차별화 혜택 제공”","한편 아이즈비전은 인공지능(AI) 기반 챗봇 서비스 도입, <b>정보보안</b> 강...",https://www.etoday.co.kr/news/view/2442493,https://www.etoday.co.kr/news/view/2442493,20250205
2,3,정보보안,LG유플러스 &quot;중고폰 고객데이터 A4용지 148억장 분량 삭제했다&quot;,이현승 LG유플러스 구독/옴니플랫폼담당(상무)은 &quot;빠르게 성장하는 중고폰 ...,https://www.todayeconomic.com/news/article.htm...,https://www.todayeconomic.com/news/article.htm...,20250205
3,4,정보보안,"아이즈비전, ‘쯔양 요금제’ 출시…“사회적 가치·차별화 혜택 제공”","한편 아이즈비전은 인공지능(AI) 기반 챗봇 서비스 도입, <b>정보보안</b> 강...",https://view.asiae.co.kr/article/2025020513252...,https://n.news.naver.com/mnews/article/277/000...,20250205
4,5,정보보안,"삼성전자, 갤S25 시리즈에 '양자 내성 암호' 기술 도입",삼성전자는 &quot;갤럭시 S25 시리즈를 시작으로 업계 최초로 PQC 기반의 클...,https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,20250205
...,...,...,...,...,...,...,...
669,53,보안업계,세계 휘저은 고래… “시장 확대” 기대 반 “데이터 독점” 우려 반[‘...,중국의 첨단 기술력에 대한 경계가 커졌지만 국내 AI <b>업계</b>는 대체로 시...,https://www.seoul.co.kr/news/plan/deepseek-sho...,https://n.news.naver.com/mnews/article/081/000...,20250205
670,54,보안업계,"&quot;금융의 미래, 디지털 자산에 있다&quot; 한국에 기대 건 파이어블록스",그는 &quot;디지털 자산에 대한 기관 참여가 심화되며 기술적 요구가 <b>보안<...,http://news.mt.co.kr/mtview.php?no=20250204164...,https://n.news.naver.com/mnews/article/008/000...,20250205
671,55,보안업계,휴면카드 1년 새 182만 장 늘었다,카드<b>업계</b> 관계자는 “휴면카드 재활성화 과정에서 본인 인증 과정을 거쳐야...,https://www.etoday.co.kr/news/view/2442217,https://www.etoday.co.kr/news/view/2442217,20250205
672,56,보안업계,중국 AI 딥시크에 대한 개인정보위의 질의,안전한 <b>보안</b> 조치를 강조하고 있다. 개인정보위는 딥시크의 답변 시기를 ...,https://www.jemin.com/news/articleView.html?id...,https://www.jemin.com/news/articleView.html?id...,20250205


In [98]:
def clean_description(text):
    text = re.sub(r"<\/?b>", "", text)  # <b>, </b> 태그 제거
    text = text.replace("&quot;", "\"")  # &quot; → " 로 변환
    return text.strip()  # 앞뒤 공백 제거

# ✅ 뉴스 데이터 로드
file_path = f"Filtered_News_{target_date}.csv"
df = pd.read_csv(file_path)
df['title'] = df["title"].apply(clean_description)
df['description'] = df["description"].apply(clean_description)


df[df['topic'] == "클라우드"]

,cnt,topic,title,description,org_link,link,pDate
98,1,클라우드,"LG CNS, 상장 첫 날 5%대 약세…시장선 '신중론'",한편 LG CNS는 이번 코스피 상장을 통해 확보한 투자 재원 중 6000억원을 인...,https://dealsite.co.kr/articles/136031,https://dealsite.co.kr/articles/136031,20250205
99,2,클라우드,"베리타스 통합한 코헤시티 ""데이터 백업 넘어 AI 분석 플랫폼으로 진화...","회사 측에 따르면 기업 데이터는 클라우드와 스토리지에 있는 파일, 폴더, 문서 등에...",https://www.digitaltoday.co.kr/news/articleVie...,https://www.digitaltoday.co.kr/news/articleVie...,20250205
100,3,클라우드,"에티버스, 에스티씨랩과 총판 계약···트래픽 매니지먼트·클라우드 사...",트래픽 제어와 클라우드 자원의 효율 관리가 필요한 시장을 공략한다는 방침이다. 에스...,https://www.etnews.com/20250205000209,https://n.news.naver.com/mnews/article/030/000...,20250205
101,4,클라우드,"IBM, 연 5% 이상 매출 증가 전망…주가 상승세 지속","하이브리드 클라우드 및 AI 전략을 강화하고 있다. 반면, 컨설팅 부문은 최근 4분...",https://www.econovill.com/news/articleView.htm...,https://www.econovill.com/news/articleView.htm...,20250205
102,5,클라우드,"네이버 이해진 창업자 경영 복귀…AI 경쟁 ""성공신화 다시 쓴다""","이를 위해 이해진 창업자와 최수연 대표는 색, 광고, 쇼핑, 클라우드 등에 AI를 ...",https://www.news1.kr/it-science/general-it/567...,https://n.news.naver.com/mnews/article/421/000...,20250205
...,...,...,...,...,...,...,...
205,108,클라우드,올트먼·이재용·손정의 3자 회동…스타게이트 동맹하나,"스타게이트 프로젝트는 미국 내 첨단 AI 인프라를 구축하기 위해 슈퍼컴퓨터, 대규모...",https://www.smartfn.co.kr/news/articleView.htm...,https://www.smartfn.co.kr/news/articleView.htm...,20250205
206,109,클라우드,"애플, 이벤트 초대장 생성 앱 출시…유료 구독 강화",애플 초대는 이날부터 앱스토어에서 내려받거나 아이클라우드를 통해 웹에서 이용할 수 ...,https://www.newsis.com/view/NISX20250205_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205
207,110,클라우드,"[EYE ON AI] 오픈AI는 ""역사의 잘못된 편""에 섰는가",포춘지의 편집장인 앨리슨 숀텔(Alyson Shontell)이 클라우드플레어(Clo...,https://www.fortunekorea.co.kr/news/articleVie...,https://www.fortunekorea.co.kr/news/articleVie...,20250205
208,111,클라우드,"알파벳, 2025년 750억 달러 투자… AI 투자 경쟁 가속화","AI 인프라 확대를 위한 대규모 투자 계획 공개 구글 클라우드 성장세 지속, AI ...",https://www.mk.co.kr/article/11232777,https://n.news.naver.com/mnews/article/009/000...,20250205


In [99]:
import pandas as pd
import re
from newspaper import Article
from fuzzywuzzy import fuzz

# ✅ 관련 키워드 리스트
related_keywords = {
    '정보보안': ['사이버', '해킹', '랜섬웨어', '보안', '위협', '데이터', '보호', '보안 사고', '시스템', '네트워크', '개인정보'],
    '클라우드': ['컴퓨팅', '서버', '데이터', '가상화', '서비스', '온라인', '인터넷', 'IT',  '인프라', '기업', '웹'],
    '제로 트러스트': ['보안', '정책', '접근', '제어', '네트워크', '보안', '인증', '데이터', '기업', '인증', '비밀번호', '권한', '관리'],
    '정보유출사고': ['데이터', '유출', '보안', '사고', '개인', '정보',  '보안', '고객', '정보', '보호', '사이버','공격', '침해'],
    '나이스': ['교육', '시스템', '학교', '관리', '행정', '학생', '정보', '교사', '업무', '교육청', '전자', '문서', '공공', 'IT', '디지털', '행정', 'NEIS'],
    '블록체인': ['디지털', '자산', '거래', '시스템', '보안', '기술', '데이터', '무결성', '탈중앙화', '네트워크', '금융', '기술', '온라인', '거래', '암호화', '블록체인'],
    'STO': ['증권', '투자', '자산', '관리', '금융', '시장', '거래', '블록체인', '금융', '기업', '투자', '디지털', '금융', '경제', 'STO'],
    'AI': ['인공지능', '기술', '데이터', '분석', '컴퓨터', '자동화', '빅데이터', '딥러닝', '소프트웨어', '산업', '혁신', 'AI'],
    '망분리': ['보안', '네트워크', '기업 보안', '인터넷', '분리', '데이터', '보호', '네트워크', '관리', '정보', '보호', '보안', '인프라', '기업', 'IT'],
    'IT업계': ['소프트웨어', '하드웨어', '기업', '스타트업', '기술 산업', '디지털', '혁신', '시장', '비즈니스', 'IT'],
    '보안업계': ['보안', '기술', '보안', '서비스', '위협', '관리', '사이버', '공격', '정보', '보호', '보안', '솔루션', '기업', '보안', '데이터', '관리']
}


# ✅ 뉴스 기사에서 키워드 등장 횟수 계산 (제목 및 주제명 가중치 적용)
def count_keywords_with_weight(text_title, text_desc, topic, keyword_list, title_weight=3, topic_weight=12, industry_weight = 3.5):
    """
    - 제목과 본문에서 키워드 등장 횟수를 계산
    - 제목에는 `title_weight`의 가중치를 적용
    - 제목에 `related_keywords`의 key (주제명)가 포함되어 있으면 `topic_weight` 추가 점수 부여
    """
    title_count = sum(text_title.count(keyword) for keyword in keyword_list)  # 제목에서 등장 횟수
    desc_count = sum(text_desc.count(keyword) for keyword in keyword_list)  # 본문에서 등장 횟수
    score = (title_count * title_weight) + desc_count  # 제목 가중치 적용

    # ✅ 제목에 `related_keywords`의 key(주제명)가 포함된 경우 추가 점수 부여
    if topic in text_title:
        score += topic_weight  # 주제명이 포함되었을 경우 추가 점수
    if "업계" in text_title or "업계" in text_desc:
        score += industry_weight

    return score

# ✅ 뉴스 본문에서 `description`과 유사도가 높은 문장 찾기
def extract_context_sentences(text, description, similarity_threshold=30):
    description = re.sub(r"<\/?b>", "", description).strip()  # <b> 태그 제거
    sentences = re.split(r'(?<=\.)\s+|\n+', text)  # 문장 분리
    matched_idx = None
    max_similarity = 0

    for idx, sent in enumerate(sentences):
        similarity = fuzz.ratio(description, sent)
        if similarity >= similarity_threshold and similarity > max_similarity:
            max_similarity = similarity
            matched_idx = idx

    if matched_idx is not None:
        start_idx = max(0, matched_idx - 1)
        end_idx = min(len(sentences) - 1, matched_idx + 1)
        imp_sent = " ".join(sentences[start_idx:end_idx + 1])
    else:
        imp_sent = "관련 문장 없음"

    return imp_sent

# ✅ 뉴스 크롤링 및 중요 내용 추출
def extract_news_data(row):
    try:
        article = Article(row['org_link'], language='ko')
        article.download()
        article.parse()

        title = article.title.strip()
        description = row['description']
        imp_sent = extract_context_sentences(article.text, description)
        if imp_sent == '관련 문장 없음':
          imp_sent = description

        return title, imp_sent
    except Exception as e:
        print(f"❌ {row['org_link']} 크롤링 오류: {e}")
        return "제목 없음", "본문 없음"

# ✅ 키워드별로 가장 관련성이 높은 뉴스 기사 선정 (주제명 포함 가중치 적용)
def select_top_news_with_weight(df):
    selected_news = []

    for topic, keywords in related_keywords.items():
        df_topic = df[df['topic'] == topic].copy()
        if df_topic.empty:
            continue

        # 키워드 등장 횟수를 기반으로 점수 계산 (제목 및 주제명 가중치 적용)
        df_topic['keyword_score'] = df_topic.apply(
            lambda row: count_keywords_with_weight(row['title'], row['description'], topic, keywords),
            axis=1
        )

        # 관련성이 높은 순으로 정렬
        df_topic = df_topic.sort_values(by='keyword_score', ascending=False)

        # 가장 관련성이 높은 뉴스 기사 1개 선택
        best_news = df_topic.iloc[0]
        selected_news.append(best_news)

    return pd.DataFrame(selected_news)


import re
def clean_description(text):
    text = re.sub(r"<\/?b>", "", text)  # <b>, </b> 태그 제거
    text = text.replace("&quot;", "\"")  # &quot; → " 로 변환
    return text.strip()  # 앞뒤 공백 제거


# ✅ 뉴스 데이터 로드
file_path = f"Filtered_News_{target_date}.csv"
df = pd.read_csv(file_path)
df['title'] = df["title"].apply(clean_description)
df['description'] = df["description"].apply(clean_description)

# ✅ 뉴스 제목과 중요 내용 추출


# ✅ 키워드별 가장 관련성이 높은 뉴스 기사 선정 (제목 및 주제명 포함 가중치 적용)
df_top_news = select_top_news_with_weight(df)
display(df_top_news)


# ✅ 중복 뉴스 제거 (제목 유사도 비교)
# ✅ 중복 뉴스 제거 (같은 제목이 있을 경우 keyword_score가 가장 높은 뉴스만 남김)
df_top_news = df_top_news.sort_values(by=['title', 'keyword_score'], ascending=[True, False])  # keyword_score가 높은 순으로 정렬
df_top_news = df_top_news.drop_duplicates(subset=['title'], keep='first')  # 같은 제목 중 keyword_score가 가장 높은 것 유지


# ✅ 최종 데이터 출력
display(df_top_news)

df_top_news[['new_title', 'imp_sent']] = df_top_news.apply(lambda row: pd.Series(extract_news_data(row)), axis=1)


display(df_top_news)

# ✅ 결과 CSV 저장
df_top_news.to_csv("Top_Related_News_Weighted_Topics_20250203.csv", encoding='utf-8-sig', index=False)
print("✅ 키워드별 가장 관련성 높은 뉴스 저장 완료 (제목 및 주제명 가중치 적용)")


,cnt,topic,title,description,org_link,link,pDate,keyword_score
8,9,정보보안,"머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까",정보 보안이 불안정한 상황에서 연방 정부 내 사이버보안 담당자들은 DOGE 팀의 활...,https://www.itworld.co.kr/article/3816968/머스크의...,https://www.itworld.co.kr/article/3816968/머스크의...,20250205,10.0
150,53,클라우드,"인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득",클라우드 EDI 전문기업 인스피언(대표 최정규)는 전개중인 ‘커넥트 서비스’가 국내...,https://www.venturesquare.net/956316,https://www.venturesquare.net/956316,20250205,22.5
210,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,10.0
211,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,19.0
243,26,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,28.0
356,86,블록체인,"소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료",소니드(060230)는 자회사 소니드에이아이가 온디바이스 인공지능(AI) ‘브레인봇...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250205,30.0
389,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,17.0
467,78,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,33.0
581,1,망분리,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...",그는 올해 주목하고 있는 변화로 망분리 제도 완화를 꼽았다. 공공·금융기관은 물리적...,https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,4.0
612,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,22.5


,cnt,topic,title,description,org_link,link,pDate,keyword_score
612,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,22.5
618,2,보안업계,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ...",http://www.sisunnews.co.kr/news/articleView.ht...,http://www.sisunnews.co.kr/news/articleView.ht...,20250205,26.5
210,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,10.0
243,26,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,28.0
8,9,정보보안,"머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까",정보 보안이 불안정한 상황에서 연방 정부 내 사이버보안 담당자들은 DOGE 팀의 활...,https://www.itworld.co.kr/article/3816968/머스크의...,https://www.itworld.co.kr/article/3816968/머스크의...,20250205,10.0
389,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,17.0
356,86,블록체인,"소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료",소니드(060230)는 자회사 소니드에이아이가 온디바이스 인공지능(AI) ‘브레인봇...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250205,30.0
150,53,클라우드,"인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득",클라우드 EDI 전문기업 인스피언(대표 최정규)는 전개중인 ‘커넥트 서비스’가 국내...,https://www.venturesquare.net/956316,https://www.venturesquare.net/956316,20250205,22.5
211,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,19.0
467,78,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,33.0


,cnt,topic,title,description,org_link,link,pDate,keyword_score,new_title,imp_sent
612,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,22.5,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...
618,2,보안업계,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ...",http://www.sisunnews.co.kr/news/articleView.ht...,http://www.sisunnews.co.kr/news/articleView.ht...,20250205,26.5,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ..."
210,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,10.0,국내 최고 자본시장(Capital Markets) 미디어,"라온시큐어 SSO 플랫폼 '터치엔 와이즈액세스' ""캐시카우가 되는 모바일 보안 사업..."
243,26,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,28.0,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추진",[자료= 경기도교육청] [포인트데일리 김장수 기자] 경기도교육정보기록원이 12일부터...
8,9,정보보안,"머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까",정보 보안이 불안정한 상황에서 연방 정부 내 사이버보안 담당자들은 DOGE 팀의 활...,https://www.itworld.co.kr/article/3816968/머스크의...,https://www.itworld.co.kr/article/3816968/머스크의...,20250205,10.0,"머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까",개럿은 “현재 이 데이터가 적절한 방식으로 백업되고 있다고 신뢰할 수 있는가? 기존...
389,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,17.0,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","규제를 준수하는 환경 속에서 실물 우라늄을 디지털 토큰으로 전환할 경우, 기존 거래..."
356,86,블록체인,"소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료",소니드(060230)는 자회사 소니드에이아이가 온디바이스 인공지능(AI) ‘브레인봇...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250205,30.0,"소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료",[이데일리 박정수 기자] 소니드(060230)는 자회사 소니드에이아이가 온디바이스 ...
150,53,클라우드,"인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득",클라우드 EDI 전문기업 인스피언(대표 최정규)는 전개중인 ‘커넥트 서비스’가 국내...,https://www.venturesquare.net/956316,https://www.venturesquare.net/956316,20250205,22.5,"인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득",클라우드 EDI 전문기업 인스피언(대표 최정규)는 전개중인 ‘커넥트 서비스’가 국내...
211,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,19.0,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","행정안전부, 개인정보보호법 등 다수 기관에서 요구하는 보안 안정성을 모두 충족한다...."
467,78,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,33.0,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""","허 원장은 ""보험산업의 데이터와 기술력이 결합해 기존 플랫폼보다 더 체계적인 데이터..."


✅ 키워드별 가장 관련성 높은 뉴스 저장 완료 (제목 및 주제명 가중치 적용)


In [100]:
# 데이터프레임에 없는 기업 목록 찾기
missing_topic = [topic for topic in related_keywords.keys() if topic not in df_top_news["topic"].values]

# 없는 기업들을 추가할 데이터프레임 생성
missing_data = pd.DataFrame({


    "cnt": 0,
    "topic": missing_topic,
    "title": "뉴스 없음",
    "description": "뉴스 없음",
    "org_link": "뉴스 없음",
    "link": "뉴스 없음",
    "pDate": int(target_date),  # 기본 날짜 설정 (필요 시 수정 가능)
    'new_title':"뉴스 없음",
    "imp_sent": "뉴스 없음",
    "keyword_score":0
})

# 기존 데이터프레임과 합치기
df_top_news = pd.concat([df_top_news, missing_data], ignore_index=True)
display(df_top_news)

,cnt,topic,title,description,org_link,link,pDate,keyword_score,new_title,imp_sent
0,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,22.5,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...
1,2,보안업계,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ...",http://www.sisunnews.co.kr/news/articleView.ht...,http://www.sisunnews.co.kr/news/articleView.ht...,20250205,26.5,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ..."
2,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,10.0,국내 최고 자본시장(Capital Markets) 미디어,"라온시큐어 SSO 플랫폼 '터치엔 와이즈액세스' ""캐시카우가 되는 모바일 보안 사업..."
3,26,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,28.0,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추진",[자료= 경기도교육청] [포인트데일리 김장수 기자] 경기도교육정보기록원이 12일부터...
4,9,정보보안,"머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까",정보 보안이 불안정한 상황에서 연방 정부 내 사이버보안 담당자들은 DOGE 팀의 활...,https://www.itworld.co.kr/article/3816968/머스크의...,https://www.itworld.co.kr/article/3816968/머스크의...,20250205,10.0,"머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까",개럿은 “현재 이 데이터가 적절한 방식으로 백업되고 있다고 신뢰할 수 있는가? 기존...
5,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,17.0,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","규제를 준수하는 환경 속에서 실물 우라늄을 디지털 토큰으로 전환할 경우, 기존 거래..."
6,86,블록체인,"소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료",소니드(060230)는 자회사 소니드에이아이가 온디바이스 인공지능(AI) ‘브레인봇...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250205,30.0,"소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료",[이데일리 박정수 기자] 소니드(060230)는 자회사 소니드에이아이가 온디바이스 ...
7,53,클라우드,"인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득",클라우드 EDI 전문기업 인스피언(대표 최정규)는 전개중인 ‘커넥트 서비스’가 국내...,https://www.venturesquare.net/956316,https://www.venturesquare.net/956316,20250205,22.5,"인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득",클라우드 EDI 전문기업 인스피언(대표 최정규)는 전개중인 ‘커넥트 서비스’가 국내...
8,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,19.0,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","행정안전부, 개인정보보호법 등 다수 기관에서 요구하는 보안 안정성을 모두 충족한다...."
9,78,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,33.0,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""","허 원장은 ""보험산업의 데이터와 기술력이 결합해 기존 플랫폼보다 더 체계적인 데이터..."


In [101]:
df_top_news["topic"] = pd.Categorical(df_top_news["topic"], categories=related_keywords.keys(), ordered=True)
df_top_news = df_top_news.sort_values("topic").reset_index(drop=True)


def replace_kbs_news_title(row):
    similarity = fuzz.partial_ratio(str(row['title']), str(row['new_title']))  # 문자열 유사도 계산
    if row['new_title'] == 'KBS 뉴스':
        return row['title']

    if similarity <= 30:
        return row['title']  # 기존 title 값으로 대체
    return row['new_title']  # 조건을 만족하지 않으면 기존 new_title 유지

df_top_news['new_title'] = df_top_news.apply(replace_kbs_news_title, axis=1)
display(df_top_news)

,cnt,topic,title,description,org_link,link,pDate,keyword_score,new_title,imp_sent
0,9,정보보안,"머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까",정보 보안이 불안정한 상황에서 연방 정부 내 사이버보안 담당자들은 DOGE 팀의 활...,https://www.itworld.co.kr/article/3816968/머스크의...,https://www.itworld.co.kr/article/3816968/머스크의...,20250205,10.0,"머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까",개럿은 “현재 이 데이터가 적절한 방식으로 백업되고 있다고 신뢰할 수 있는가? 기존...
1,53,클라우드,"인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득",클라우드 EDI 전문기업 인스피언(대표 최정규)는 전개중인 ‘커넥트 서비스’가 국내...,https://www.venturesquare.net/956316,https://www.venturesquare.net/956316,20250205,22.5,"인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득",클라우드 EDI 전문기업 인스피언(대표 최정규)는 전개중인 ‘커넥트 서비스’가 국내...
2,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,10.0,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","라온시큐어 SSO 플랫폼 '터치엔 와이즈액세스' ""캐시카우가 되는 모바일 보안 사업..."
3,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,19.0,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","행정안전부, 개인정보보호법 등 다수 기관에서 요구하는 보안 안정성을 모두 충족한다...."
4,26,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,28.0,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추진",[자료= 경기도교육청] [포인트데일리 김장수 기자] 경기도교육정보기록원이 12일부터...
5,86,블록체인,"소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료",소니드(060230)는 자회사 소니드에이아이가 온디바이스 인공지능(AI) ‘브레인봇...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250205,30.0,"소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료",[이데일리 박정수 기자] 소니드(060230)는 자회사 소니드에이아이가 온디바이스 ...
6,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,17.0,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","규제를 준수하는 환경 속에서 실물 우라늄을 디지털 토큰으로 전환할 경우, 기존 거래..."
7,78,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,33.0,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""","허 원장은 ""보험산업의 데이터와 기술력이 결합해 기존 플랫폼보다 더 체계적인 데이터..."
8,0,망분리,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250205,0.0,뉴스 없음,뉴스 없음
9,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,22.5,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...


In [102]:
for i in range(len(df_top_news)):
  print(df_top_news['new_title'][i])

머스크의 DOGE 프로젝트, 미국 사이버보안에 재앙 불러올까
인스피언 ‘커넥트 서비스’, 클라우드 보안인증 획득
[더벨][퀀텀점프 2025]라온시큐어 "인증 기술 토대 신사업 전개, 세계 무...
인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료
경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추진
소니드에이아이, 블록체인 보안기술 적용 AI CCTV 솔루션 개발 완료
블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대
허창언 보험개발원장 "빅데이터와 AI 활용 보험산업 플랫폼 강화할 것"
뉴스 없음
'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자'
[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ‘넷킬러(NETKILLER)’


In [103]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment

# 엑셀 파일 경로
excel_path = f"/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"
new_excel_path =  f"SGA업계 및 경쟁사 현황_수치_공시_뉴스{target_date}.xlsx"
sheet_name = "업계현황"
output_sheet_name = "업계현황"  # 새로운 시트 이름

# 데이터프레임 로드 (bs_info_df, CIS_info_df는 미리 정의되어 있어야 함)
# 예시로 df를 bs_info_df로 지정합니다. 필요에 따라 변경하세요.


# 기존 엑셀 파일 불러오기 (서식 유지)
wb = load_workbook(excel_path)
ws_old = wb[sheet_name]

# 시트 이름이 기존에 존재하는지 확인
if output_sheet_name in wb.sheetnames:
    ws = wb[output_sheet_name]  # 기존 시트를 선택
    print(f"기존 시트 '{output_sheet_name}'에 데이터를 추가합니다.")
else:
    ws = wb.copy_worksheet(ws_old)  # 기존 시트를 복제
    ws.title = output_sheet_name  # 복제한 시트의 이름 변경
    print(f"새로운 시트 '{output_sheet_name}'을(를) 생성하였습니다.")

# 엑셀의 3행부터 14행까지 있는 회사명 찾기
target_rows = range(3, 17)  # openpyxl은 1-based index (pandas는 0-based index)
for row in target_rows:
    company_name = ws.cell(row=row, column=1).value  # 회사명 가져오기
    if company_name in df_top_news['topic'].values:
        # 'new_title' 값 가져오기
        title_value = df_top_news.loc[df_top_news['topic'] == company_name,'new_title'].values[0]
        if title_value == '뉴스 없음':
            print(f"'{company_name}'의 'new_title'이 '제목 없음'이므로 건너뜁니다.")
            continue  # 'new_title'이 '제목 없음'이면 해당 행 건너뛰기
        # 'new_title' 값을 2열에 추가
        title_cell = ws.cell(row=row, column=2, value=title_value)
        # 'org_link'를 하이퍼링크로 추가
        org_link_value = df_top_news.loc[df_top_news['topic'] == company_name, 'org_link'].values[0]
        title_cell.hyperlink = org_link_value
        title_cell.font = Font(color="0000FF", underline="single")
        # 'imp_sent' 값을 3열에 추가
        imp_sent_value = df_top_news.loc[df_top_news['topic'] == company_name, 'imp_sent'].values[0]
        sentence_cell = ws.cell(row=row, column=3, value=imp_sent_value)
        # 셀 서식 설정 (예: 글꼴 굵게, 가운데 정렬)

        title_cell.alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')
        sentence_cell.alignment = Alignment(wrap_text=True,horizontal='center', vertical='center')
    else:
        print(f"데이터프레임에 '{company_name}'에 대한 정보가 없습니다.")

# 엑셀 파일 저장
wb.save(new_excel_path)
print(f"데이터프레임이 '{new_excel_path}' 파일에 성공적으로 추가되었습니다.")


기존 시트 '업계현황'에 데이터를 추가합니다.
데이터프레임에 '제로트러스트'에 대한 정보가 없습니다.
'망분리'의 'new_title'이 '제목 없음'이므로 건너뜁니다.
데이터프레임에 '기타'에 대한 정보가 없습니다.
데이터프레임에 'None'에 대한 정보가 없습니다.
데이터프레임에 'None'에 대한 정보가 없습니다.
데이터프레임이 'SGA업계 및 경쟁사 현황_수치_공시_뉴스20250205.xlsx' 파일에 성공적으로 추가되었습니다.


# 코드

In [104]:
import urllib.request
import datetime
import json
import pandas as pd
from urllib.parse import quote

# 네이버 API 인증 정보
# 네이버 API 인증 정보
client_id = "IamYBKISQmZU5MdHRnsm"
client_secret = "AEZKqyZhux"


# 검색할 주제와 관련 키워드 리스트
related_keywords = {
    '정보보안': ['사이버', '해킹', '랜섬웨어', '보안', '위협', '데이터', '보호', '보안 사고', '시스템', '네트워크', '개인정보', '정보보안'],
    '클라우드': ['컴퓨팅', '서버', '데이터', '가상화', '서비스', '온라인', '인터넷', 'IT',  '인프라', '기업', '웹'],
    '제로 트러스트': ['보안', '정책', '접근', '제어', '네트워크', '보안', '인증', '데이터', '기업', '인증', '비밀번호', '권한', '관리'],
    '정보유출사고': ['데이터 유출', '보안 사고', '개인정보', '기업 보안', '고객 정보', '정보 보호', '정보 유출', '사이버 공격', '데이터 침해'],
    '나이스': ['교육', '시스템', '학교', '관리', '행정', '학생', '정보', '교사', '업무', '교육청', '전자', '문서', '공공', 'IT', '디지털', '행정', 'NEIS', '나이스'],
    '블록체인': ['디지털', '자산', '거래', '시스템', '보안', '기술', '데이터', '무결성', '탈중앙화', '네트워크', '금융', '기술', '온라인', '거래', '암호화', '블록체인'],
    'STO': ['증권', '투자', '자산', '관리', '금융', '시장', '거래', '블록체인', '금융', '기업', '투자', '디지털', '금융', '경제', 'STO'],
    'AI': ['인공지능', '기술', '데이터', '분석', '컴퓨터', '자동화', '빅데이터', '딥러닝', '소프트웨어', '산업', '혁신', 'AI'],
    '망분리': ['보안', '네트워크', '기업 보안', '인터넷', '분리', '데이터', '보호', '네트워크', '관리', '정보', '보호', '보안', '인프라', '기업', 'IT', '망분리'],
    'IT업계': ['소프트웨어', '하드웨어', '기업', '스타트업', '기술 산업', '디지털', '혁신', '시장', '비즈니스', 'IT'],
    '보안업계': ['보안', '기술', '보안', '서비스', '위협', '관리', '사이버', '공격', '정보', '보호', '보안', '솔루션', '기업', '보안', '데이터', '관리']
}
# 제외할 키워드 리스트 (불필요한 뉴스 필터링)
EXCLUDE_KEYWORDS = ["연예", "스포츠", "날씨", "범죄", "게임", "드라마", "영화", "NBA", "축제", "추위", "호텔", "얼음", "가수", "아이돌", "축제", "아카데미"]

# 네이버 API 요청 함수
def get_request_url(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# 네이버 뉴스 검색 API 호출 함수 (최대 100개만 가져오기)
def get_naver_search(keyword):
    base = "https://openapi.naver.com/v1/search/news.json"
    parameters = f"?query={quote(keyword)}&start=1&display=100&sort=date"  # 최신순 정렬
    url = base + parameters

    response = get_request_url(url)
    if response is None:
        return None
    else:
        return json.loads(response)

# 뉴스 데이터 필터링 함수
def filter_news(post, keywords):
    """
    기사 제목이나 설명에 주어진 키워드 리스트의 단어가 포함되고,
    EXCLUDE_KEYWORDS(제외할 키워드)가 포함되지 않은 경우만 True 반환
    """
    print()
    title = post['title']
    description = post['description']

    # 포함할 키워드 체크 (하나라도 포함되면 OK)
    if not any(keyword in title or keyword in description for keyword in keywords):
        return False  # 관련 키워드가 포함되지 않은 기사

    # 제외할 키워드 체크 (하나라도 포함되면 False)
    if any(keyword in title or keyword in description for keyword in EXCLUDE_KEYWORDS):
        return False  # 제외할 키워드가 포함된 기사이므로 필터링

    return True  # 관련 키워드가 포함되며, 제외할 키워드가 포함되지 않은 경우

# 검색 실행 함수
def main():

    all_news = []  # 모든 주제의 뉴스를 저장할 리스트
    total_cnt = 0  # 전체 뉴스 개수 카운트

    for topic, keywords in related_keywords.items():
        json_result = []
        cnt = 0

        # API 요청
        json_response = get_naver_search(topic)

        if json_response is None or 'items' not in json_response:
            print(f"❌ '{topic}'에 대한 뉴스 검색 결과가 없습니다.")
            continue

        total = json_response['total']
        print(f"🔎 '{topic}' 전체 검색 결과: {total}건 (최대 100개 검색)")

        # 뉴스 가져오기 및 날짜 + 키워드 필터링
        for post in json_response['items']:
            # 날짜 변환 (YYYYMMDD)
            p_date = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
            p_date = p_date.strftime('%Y%m%d')

            # 입력한 날짜와 일치하고, 관련 키워드 포함 & 불필요한 뉴스 제외
            if p_date == target_date and filter_news(post, keywords):
                cnt += 1
                json_result.append({
                    'cnt': cnt,
                    'topic': topic,  # 주제명 추가
                    'title': post['title'],
                    'description': post['description'],
                    'org_link': post['originallink'],
                    'link': post['link'],
                    'pDate': p_date
                })

        # 필터링된 뉴스만 리스트에 추가
        all_news.extend(json_result)
        total_cnt += len(json_result)

    # JSON 저장
    filename = f"Filtered_News_{target_date}.json"
    with open(filename, 'w', encoding='utf8') as outfile:
        json.dump(all_news, outfile, indent=4, sort_keys=True, ensure_ascii=False)

    print(f"✅ 총 가져온 데이터: {total_cnt}건")
    print(f"📂 저장 완료: {filename}")

    # 결과를 DataFrame으로 변환하여 CSV 저장
    df = pd.DataFrame(all_news)
    if not df.empty:
        df.to_csv(f"Filtered_News_{target_date}.csv", encoding='utf-8-sig', index=False)
        print(f"📂 CSV 저장 완료: Filtered_News_{target_date}.csv")
        display(df)
    else:
        print("❌ 해당 날짜에 대한 관련 뉴스가 없습니다.")

if __name__ == '__main__':
    main()


🔎 '정보보안' 전체 검색 결과: 338243건 (최대 100개 검색)




































































































🔎 '클라우드' 전체 검색 결과: 1225332건 (최대 100개 검색)




































































































🔎 '제로 트러스트' 전체 검색 결과: 17858건 (최대 100개 검색)

🔎 '정보유출사고' 전체 검색 결과: 149018건 (최대 100개 검색)









🔎 '나이스' 전체 검색 결과: 199211건 (최대 100개 검색)
































































🔎 '블록체인' 전체 검색 결과: 967365건 (최대 100개 검색)




































































































🔎 'STO' 전체 검색 결과: 24313건 (최대 100개 검색)






🔎 'AI' 전체 검색 결과: 3836687건 (최대 100개 검색)




































































































🔎 '망분리' 전체 검색 결과: 18231건 (최대 100개 검색)

🔎 'IT업계' 전체 검색 결과: 186254건 (최대 100개 검색)




































🔎 '보안업계' 전체 검색 결과: 315502건 (최대 100개 검색)














































,cnt,topic,title,description,org_link,link,pDate
0,1,정보보안,"LG유플러스, 중고폰 속 고객데이터 A4용지 148억장 분량 삭제했다",이현승 LG유플러스 구독/옴니플랫폼담당(상무)은 “빠르게 성장하는 중고폰 시장에서 ...,https://www.swtvnews.com/news/newsview.php?nco...,https://www.swtvnews.com/news/newsview.php?nco...,20250205
1,2,정보보안,"슈프리마, '바이오스타 2 VMS' GS인증 1등급 획득","한국인터넷진흥원, 한국정보보호산업협회 -일시: 2025년 2월 11일 화요일 오전 ...",https://www.dailysecu.com/news/articleView.htm...,https://www.dailysecu.com/news/articleView.htm...,20250205
2,3,정보보안,"아이즈비전, ‘쯔양 요금제’ 출시…“사회적 가치ㆍ차별화 혜택 제공”","한편 아이즈비전은 인공지능(AI) 기반 챗봇 서비스 도입, <b>정보보안</b> 강...",https://www.etoday.co.kr/news/view/2442493,https://www.etoday.co.kr/news/view/2442493,20250205
3,4,정보보안,LG유플러스 &quot;중고폰 고객데이터 A4용지 148억장 분량 삭제했다&quot;,이현승 LG유플러스 구독/옴니플랫폼담당(상무)은 &quot;빠르게 성장하는 중고폰 ...,https://www.todayeconomic.com/news/article.htm...,https://www.todayeconomic.com/news/article.htm...,20250205
4,5,정보보안,"아이즈비전, ‘쯔양 요금제’ 출시…“사회적 가치·차별화 혜택 제공”","한편 아이즈비전은 인공지능(AI) 기반 챗봇 서비스 도입, <b>정보보안</b> 강...",https://view.asiae.co.kr/article/2025020513252...,https://n.news.naver.com/mnews/article/277/000...,20250205
...,...,...,...,...,...,...,...
499,53,보안업계,세계 휘저은 고래… “시장 확대” 기대 반 “데이터 독점” 우려 반[‘...,중국의 첨단 기술력에 대한 경계가 커졌지만 국내 AI <b>업계</b>는 대체로 시...,https://www.seoul.co.kr/news/plan/deepseek-sho...,https://n.news.naver.com/mnews/article/081/000...,20250205
500,54,보안업계,"&quot;금융의 미래, 디지털 자산에 있다&quot; 한국에 기대 건 파이어블록스",그는 &quot;디지털 자산에 대한 기관 참여가 심화되며 기술적 요구가 <b>보안<...,http://news.mt.co.kr/mtview.php?no=20250204164...,https://n.news.naver.com/mnews/article/008/000...,20250205
501,55,보안업계,휴면카드 1년 새 182만 장 늘었다,카드<b>업계</b> 관계자는 “휴면카드 재활성화 과정에서 본인 인증 과정을 거쳐야...,https://www.etoday.co.kr/news/view/2442217,https://www.etoday.co.kr/news/view/2442217,20250205
502,56,보안업계,중국 AI 딥시크에 대한 개인정보위의 질의,안전한 <b>보안</b> 조치를 강조하고 있다. 개인정보위는 딥시크의 답변 시기를 ...,https://www.jemin.com/news/articleView.html?id...,https://www.jemin.com/news/articleView.html?id...,20250205


In [105]:
import pandas as pd
import re
from newspaper import Article
from fuzzywuzzy import fuzz

# ✅ 관련 키워드 리스트
related_keywords = {
    '정보보안': ['사이버', '해킹', '랜섬웨어', '보안', '위협', '데이터', '보호', '보안 사고', '시스템', '네트워크', '개인정보'],
    '클라우드': ['컴퓨팅', '서버', '데이터', '가상화', '서비스', '온라인', '인터넷', 'IT',  '인프라', '기업', '웹'],
    '제로 트러스트': ['보안', '정책', '접근', '제어', '네트워크', '보안', '인증', '데이터', '기업', '인증', '비밀번호', '권한', '관리'],
    '정보유출사고': ['데이터', '유출', '보안', '사고', '개인', '정보',  '보안', '고객', '정보', '보호', '사이버','공격', '침해'],
    '나이스': ['교육', '시스템', '학교', '관리', '행정', '학생', '정보', '교사', '업무', '교육청', '전자', '문서', '공공', 'IT', '디지털', '행정', 'NEIS', '나이스'],
    '블록체인': ['디지털', '자산', '거래', '시스템', '보안', '기술', '데이터', '무결성', '탈중앙화', '네트워크', '금융', '기술', '온라인', '거래', '암호화', '블록체인'],
    'STO': ['증권', '투자', '자산', '관리', '금융', '시장', '거래', '블록체인', '금융', '기업', '투자', '디지털', '금융', '경제', 'STO'],
    'AI': ['인공지능', '기술', '데이터', '분석', '컴퓨터', '자동화', '빅데이터', '딥러닝', '소프트웨어', '산업', '혁신', 'AI'],
    '망분리': ['보안', '네트워크', '기업 보안', '인터넷', '분리', '데이터', '보호', '네트워크', '관리', '정보', '보호', '보안', '인프라', '기업', 'IT'],
    'IT업계': ['소프트웨어', '하드웨어', '기업', '스타트업', '기술 산업', '디지털', '혁신', '시장', '비즈니스', 'IT'],
    '보안업계': ['보안', '기술', '보안', '서비스', '위협', '관리', '사이버', '공격', '정보', '보호', '보안', '솔루션', '기업', '보안', '데이터', '관리']
}


# ✅ 뉴스 기사에서 키워드 등장 횟수 계산 (제목 및 주제명 가중치 적용)
def count_keywords_with_weight(text_title, text_desc, topic, keyword_list, title_weight=6, topic_weight=2, industry_weight = 3.5):
    """
    - 제목과 본문에서 키워드 등장 횟수를 계산
    - 제목에는 `title_weight`의 가중치를 적용
    - 제목에 `related_keywords`의 key (주제명)가 포함되어 있으면 `topic_weight` 추가 점수 부여
    """
    title_count = sum(text_title.count(keyword) for keyword in keyword_list)  # 제목에서 등장 횟수
    desc_count = sum(text_desc.count(keyword) for keyword in keyword_list)  # 본문에서 등장 횟수
    score = (title_count * title_weight) + desc_count  # 제목 가중치 적용

    # ✅ 제목에 `related_keywords`의 key(주제명)가 포함된 경우 추가 점수 부여
    if topic in text_title:
        score += topic_weight  # 주제명이 포함되었을 경우 추가 점수
    if "업계" in text_title or "업계" in text_desc:
        score += industry_weight

    return score

# ✅ 뉴스 본문에서 `description`과 유사도가 높은 문장 찾기
def extract_context_sentences(text, description, similarity_threshold=30):
    description = re.sub(r"<\/?b>", "", description).strip()  # <b> 태그 제거
    sentences = re.split(r'(?<=\.)\s+|\n+', text)  # 문장 분리
    matched_idx = None
    max_similarity = 0

    for idx, sent in enumerate(sentences):
        similarity = fuzz.ratio(description, sent)
        if similarity >= similarity_threshold and similarity > max_similarity:
            max_similarity = similarity
            matched_idx = idx

    if matched_idx is not None:
        start_idx = max(0, matched_idx - 1)
        end_idx = min(len(sentences) - 1, matched_idx + 1)
        imp_sent = " ".join(sentences[start_idx:end_idx + 1])
    else:
        imp_sent = "관련 문장 없음"

    return imp_sent

# ✅ 뉴스 크롤링 및 중요 내용 추출
def extract_news_data(row):
    try:
        article = Article(row['org_link'], language='ko')
        article.download()
        article.parse()

        title = article.title.strip()
        description = row['description']
        imp_sent = extract_context_sentences(article.text, description)
        if imp_sent == '관련 문장 없음':
          imp_sent = description

        return title, imp_sent
    except Exception as e:
        print(f"❌ {row['org_link']} 크롤링 오류: {e}")
        return "제목 없음", "본문 없음"

# ✅ 키워드별로 가장 관련성이 높은 뉴스 기사 선정 (주제명 포함 가중치 적용)
def select_top_news_with_weight(df, title_weight=2, topic_weight=3):
    selected_news = []

    for topic, keywords in related_keywords.items():
        df_topic = df[df['topic'] == topic].copy()
        if df_topic.empty:
            continue

        # 키워드 등장 횟수를 기반으로 점수 계산 (제목 및 주제명 가중치 적용)
        df_topic['keyword_score'] = df_topic.apply(
            lambda row: count_keywords_with_weight(row['title'], row['description'], topic, keywords, title_weight, topic_weight),
            axis=1
        )

        # 관련성이 높은 순으로 정렬
        df_topic = df_topic.sort_values(by='keyword_score', ascending=False)

        # 가장 관련성이 높은 뉴스 기사 1개 선택
        best_news = df_topic.iloc[0]
        selected_news.append(best_news)

    return pd.DataFrame(selected_news)


import re
def clean_description(text):
    text = re.sub(r"<\/?b>", "", text)  # <b>, </b> 태그 제거
    text = text.replace("&quot;", "\"")  # &quot; → " 로 변환
    return text.strip()  # 앞뒤 공백 제거


# ✅ 뉴스 데이터 로드
file_path = f"Filtered_News_{target_date}.csv"
df = pd.read_csv(file_path)
df['title'] = df["title"].apply(clean_description)
df['description'] = df["description"].apply(clean_description)

# ✅ 뉴스 제목과 중요 내용 추출


# ✅ 키워드별 가장 관련성이 높은 뉴스 기사 선정 (제목 및 주제명 포함 가중치 적용)
df_top_news = select_top_news_with_weight(df, title_weight=2, topic_weight=3)



# ✅ 중복 뉴스 제거 (제목 유사도 비교)
# ✅ 중복 뉴스 제거 (같은 제목이 있을 경우 keyword_score가 가장 높은 뉴스만 남김)
df_top_news = df_top_news.sort_values(by=['title', 'keyword_score'], ascending=[True, False])  # keyword_score가 높은 순으로 정렬
df_top_news = df_top_news.drop_duplicates(subset=['title'], keep='first')  # 같은 제목 중 keyword_score가 가장 높은 것 유지


# ✅ 최종 데이터 출력
display(df_top_news)

df_top_news[['new_title', 'imp_sent']] = df_top_news.apply(lambda row: pd.Series(extract_news_data(row)), axis=1)


display(df_top_news)

# ✅ 결과 CSV 저장
df_top_news.to_csv("Top_Related_News_Weighted_Topics_20250203.csv", encoding='utf-8-sig', index=False)
print("✅ 키워드별 가장 관련성 높은 뉴스 저장 완료 (제목 및 주제명 가중치 적용)")


,cnt,topic,title,description,org_link,link,pDate,keyword_score
442,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,12.5
448,2,보안업계,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ...",http://www.sisunnews.co.kr/news/articleView.ht...,http://www.sisunnews.co.kr/news/articleView.ht...,20250205,21.5
146,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,8.0
307,90,블록체인,"[최기일의 방산보국㉘] ""방위산업 분야 블록체인 플랫폼 구축해야""",◇블록체인 기술 정의와 특징적 요인에 기인한 국방 및 방위산업 활용가치=블록체인(B...,https://www.asiaa.co.kr/news/articleView.html?...,https://www.asiaa.co.kr/news/articleView.html?...,20250205,17.0
182,29,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,20.0
314,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,15.0
95,96,정보보안,"삼성전자, 갤럭시 S25에 '포스트 양자암호' 기술 도입","삼성전자는 ""갤럭시 S25 시리즈를 시작으로 업계 최초로 PQC 기반의 클라우드 데...",https://www.digitaltoday.co.kr/news/articleVie...,https://www.digitaltoday.co.kr/news/articleVie...,20250205,8.5
147,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,18.0
122,25,클라우드,"통신업계 돌파구 AI, 어떻게 잡을까- ①데이터센터","데이터센터, 클라우드, B2C AI 서비스 등을 3대 전략 축으로 설정했다. AI ...",http://www.biztribune.co.kr/news/articleView.h...,http://www.biztribune.co.kr/news/articleView.h...,20250205,11.5
394,80,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,20.0


,cnt,topic,title,description,org_link,link,pDate,keyword_score,new_title,imp_sent
442,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,12.5,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...
448,2,보안업계,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ...",http://www.sisunnews.co.kr/news/articleView.ht...,http://www.sisunnews.co.kr/news/articleView.ht...,20250205,21.5,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ..."
146,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,8.0,국내 최고 자본시장(Capital Markets) 미디어,"라온시큐어 SSO 플랫폼 '터치엔 와이즈액세스' ""캐시카우가 되는 모바일 보안 사업..."
307,90,블록체인,"[최기일의 방산보국㉘] ""방위산업 분야 블록체인 플랫폼 구축해야""",◇블록체인 기술 정의와 특징적 요인에 기인한 국방 및 방위산업 활용가치=블록체인(B...,https://www.asiaa.co.kr/news/articleView.html?...,https://www.asiaa.co.kr/news/articleView.html?...,20250205,17.0,"[최기일의 방산보국㉘] ""방위산업 분야 블록체인 플랫폼 구축해야"" - 아시아에이",새로운 시대의 표준과 기준을 뜻하는 뉴 노멀(New Normal)로서 장차 안보와 ...
182,29,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,20.0,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추진",[자료= 경기도교육청] [포인트데일리 김장수 기자] 경기도교육정보기록원이 12일부터...
314,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,15.0,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","규제를 준수하는 환경 속에서 실물 우라늄을 디지털 토큰으로 전환할 경우, 기존 거래..."
95,96,정보보안,"삼성전자, 갤럭시 S25에 '포스트 양자암호' 기술 도입","삼성전자는 ""갤럭시 S25 시리즈를 시작으로 업계 최초로 PQC 기반의 클라우드 데...",https://www.digitaltoday.co.kr/news/articleVie...,https://www.digitaltoday.co.kr/news/articleVie...,20250205,8.5,"삼성전자, 갤럭시 S25에 '포스트 양자암호' 기술 도입",One UI 7을 탑재한 갤럭시 S25 시리즈가 업계 최초로 PQC 기반 클라우드 ...
147,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,18.0,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","행정안전부, 개인정보보호법 등 다수 기관에서 요구하는 보안 안정성을 모두 충족한다...."
122,25,클라우드,"통신업계 돌파구 AI, 어떻게 잡을까- ①데이터센터","데이터센터, 클라우드, B2C AI 서비스 등을 3대 전략 축으로 설정했다. AI ...",http://www.biztribune.co.kr/news/articleView.h...,http://www.biztribune.co.kr/news/articleView.h...,20250205,11.5,"통신업계 돌파구 AI, 어떻게 잡을까- ①데이터센터","■AI 시대, 필수 인프라로 부상한 데이터센터 AI 혁신과 클라우드 컴퓨팅 확산으로..."
394,80,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,20.0,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""","허 원장은 ""보험산업의 데이터와 기술력이 결합해 기존 플랫폼보다 더 체계적인 데이터..."


✅ 키워드별 가장 관련성 높은 뉴스 저장 완료 (제목 및 주제명 가중치 적용)


In [106]:
# 데이터프레임에 없는 기업 목록 찾기
missing_topic = [topic for topic in related_keywords.keys() if topic not in df_top_news["topic"].values]

# 없는 기업들을 추가할 데이터프레임 생성
missing_data = pd.DataFrame({


    "cnt": 0,
    "topic": missing_topic,
    "title": "뉴스 없음",
    "description": "뉴스 없음",
    "org_link": "뉴스 없음",
    "link": "뉴스 없음",
    "pDate": int(target_date),  # 기본 날짜 설정 (필요 시 수정 가능)
    'new_title':"뉴스 없음",
    "imp_sent": "뉴스 없음",
    "keyword_score":0
})

# 기존 데이터프레임과 합치기
df_top_news = pd.concat([df_top_news, missing_data], ignore_index=True)
display(df_top_news)

,cnt,topic,title,description,org_link,link,pDate,keyword_score,new_title,imp_sent
0,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,12.5,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...
1,2,보안업계,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ...",http://www.sisunnews.co.kr/news/articleView.ht...,http://www.sisunnews.co.kr/news/articleView.ht...,20250205,21.5,"[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ...","이는 보안 업계에서 ""Great Reset""이라고 불리는 새로운 패러다임의 시작을 ..."
2,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,8.0,국내 최고 자본시장(Capital Markets) 미디어,"라온시큐어 SSO 플랫폼 '터치엔 와이즈액세스' ""캐시카우가 되는 모바일 보안 사업..."
3,90,블록체인,"[최기일의 방산보국㉘] ""방위산업 분야 블록체인 플랫폼 구축해야""",◇블록체인 기술 정의와 특징적 요인에 기인한 국방 및 방위산업 활용가치=블록체인(B...,https://www.asiaa.co.kr/news/articleView.html?...,https://www.asiaa.co.kr/news/articleView.html?...,20250205,17.0,"[최기일의 방산보국㉘] ""방위산업 분야 블록체인 플랫폼 구축해야"" - 아시아에이",새로운 시대의 표준과 기준을 뜻하는 뉴 노멀(New Normal)로서 장차 안보와 ...
4,29,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,20.0,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추진",[자료= 경기도교육청] [포인트데일리 김장수 기자] 경기도교육정보기록원이 12일부터...
5,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,15.0,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","규제를 준수하는 환경 속에서 실물 우라늄을 디지털 토큰으로 전환할 경우, 기존 거래..."
6,96,정보보안,"삼성전자, 갤럭시 S25에 '포스트 양자암호' 기술 도입","삼성전자는 ""갤럭시 S25 시리즈를 시작으로 업계 최초로 PQC 기반의 클라우드 데...",https://www.digitaltoday.co.kr/news/articleVie...,https://www.digitaltoday.co.kr/news/articleVie...,20250205,8.5,"삼성전자, 갤럭시 S25에 '포스트 양자암호' 기술 도입",One UI 7을 탑재한 갤럭시 S25 시리즈가 업계 최초로 PQC 기반 클라우드 ...
7,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,18.0,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","행정안전부, 개인정보보호법 등 다수 기관에서 요구하는 보안 안정성을 모두 충족한다...."
8,25,클라우드,"통신업계 돌파구 AI, 어떻게 잡을까- ①데이터센터","데이터센터, 클라우드, B2C AI 서비스 등을 3대 전략 축으로 설정했다. AI ...",http://www.biztribune.co.kr/news/articleView.h...,http://www.biztribune.co.kr/news/articleView.h...,20250205,11.5,"통신업계 돌파구 AI, 어떻게 잡을까- ①데이터센터","■AI 시대, 필수 인프라로 부상한 데이터센터 AI 혁신과 클라우드 컴퓨팅 확산으로..."
9,80,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,20.0,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""","허 원장은 ""보험산업의 데이터와 기술력이 결합해 기존 플랫폼보다 더 체계적인 데이터..."


In [107]:
df_top_news["topic"] = pd.Categorical(df_top_news["topic"], categories=related_keywords.keys(), ordered=True)
df_top_news = df_top_news.sort_values("topic").reset_index(drop=True)


def replace_kbs_news_title(row):
    similarity = fuzz.partial_ratio(str(row['title']), str(row['new_title']))  # 문자열 유사도 계산
    if row['new_title'] == 'KBS 뉴스':
        return row['title']

    if similarity <= 30:
        return row['title']  # 기존 title 값으로 대체
    return row['new_title']  # 조건을 만족하지 않으면 기존 new_title 유지

df_top_news['new_title'] = df_top_news.apply(replace_kbs_news_title, axis=1)
display(df_top_news)

,cnt,topic,title,description,org_link,link,pDate,keyword_score,new_title,imp_sent
0,96,정보보안,"삼성전자, 갤럭시 S25에 '포스트 양자암호' 기술 도입","삼성전자는 ""갤럭시 S25 시리즈를 시작으로 업계 최초로 PQC 기반의 클라우드 데...",https://www.digitaltoday.co.kr/news/articleVie...,https://www.digitaltoday.co.kr/news/articleVie...,20250205,8.5,"삼성전자, 갤럭시 S25에 '포스트 양자암호' 기술 도입",One UI 7을 탑재한 갤럭시 S25 시리즈가 업계 최초로 PQC 기반 클라우드 ...
1,25,클라우드,"통신업계 돌파구 AI, 어떻게 잡을까- ①데이터센터","데이터센터, 클라우드, B2C AI 서비스 등을 3대 전략 축으로 설정했다. AI ...",http://www.biztribune.co.kr/news/articleView.h...,http://www.biztribune.co.kr/news/articleView.h...,20250205,11.5,"통신업계 돌파구 AI, 어떻게 잡을까- ①데이터센터","■AI 시대, 필수 인프라로 부상한 데이터센터 AI 혁신과 클라우드 컴퓨팅 확산으로..."
2,1,제로 트러스트,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","""캐시카우가 되는 모바일 보안 사업을 탄탄하게 가져가고, 제로 트러스트와 양자보안,...",https://www.thebell.co.kr/free/content/Article...,https://www.thebell.co.kr/free/content/Article...,20250205,8.0,"[더벨][퀀텀점프 2025]라온시큐어 ""인증 기술 토대 신사업 전개, 세계 무...","라온시큐어 SSO 플랫폼 '터치엔 와이즈액세스' ""캐시카우가 되는 모바일 보안 사업..."
3,1,정보유출사고,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","개인정보 검출 및 점검을 통해 정보 유출 사고를 사전 예방할 수 있고, 개인 정보 ...",https://www.ddaily.co.kr/page/view/20250205104...,https://n.news.naver.com/mnews/article/138/000...,20250205,18.0,"인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료","행정안전부, 개인정보보호법 등 다수 기관에서 요구하는 보안 안정성을 모두 충족한다...."
4,29,나이스,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추...",'2025년도 나이스 교무업무 학년초 사용자 연수' 추진 유튜브. [자료= 경기도...,https://www.pointdaily.co.kr/news/articleView....,https://www.pointdaily.co.kr/news/articleView....,20250205,20.0,"경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추진",[자료= 경기도교육청] [포인트데일리 김장수 기자] 경기도교육정보기록원이 12일부터...
5,90,블록체인,"[최기일의 방산보국㉘] ""방위산업 분야 블록체인 플랫폼 구축해야""",◇블록체인 기술 정의와 특징적 요인에 기인한 국방 및 방위산업 활용가치=블록체인(B...,https://www.asiaa.co.kr/news/articleView.html?...,https://www.asiaa.co.kr/news/articleView.html?...,20250205,17.0,"[최기일의 방산보국㉘] ""방위산업 분야 블록체인 플랫폼 구축해야"" - 아시아에이",새로운 시대의 표준과 기준을 뜻하는 뉴 노멀(New Normal)로서 장차 안보와 ...
6,6,STO,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","2019년, 금융 시장에서는 ‘증권형 토큰 발행’(STO)에 대한 기대감이 컸다. ...",https://www.tokenpost.kr/article-218442,https://www.tokenpost.kr/article-218442,20250205,15.0,"블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대","규제를 준수하는 환경 속에서 실물 우라늄을 디지털 토큰으로 전환할 경우, 기존 거래..."
7,80,AI,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""",보험개발원의 대외 소통 채널인 보험정보 빅데이터 플랫폼 ‘BIGIN(비긴)’에도 빅...,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,20250205,20.0,"허창언 보험개발원장 ""빅데이터와 AI 활용 보험산업 플랫폼 강화할 것""","허 원장은 ""보험산업의 데이터와 기술력이 결합해 기존 플랫폼보다 더 체계적인 데이터..."
8,0,망분리,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250205,0.0,뉴스 없음,뉴스 없음
9,31,IT업계,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...,https://www.newsis.com/view/NISX20250204_00030...,https://n.news.naver.com/mnews/article/003/001...,20250205,12.5,'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자',최근 IT 채용이 침체된 가운데 유일하게 '인공지능(AI) 개발자'에 대한 시장의 ...


In [108]:
for i in range(len(df_top_news)):
  print(df_top_news['new_title'][i])

삼성전자, 갤럭시 S25에 '포스트 양자암호' 기술 도입
통신업계 돌파구 AI, 어떻게 잡을까- ①데이터센터
[더벨][퀀텀점프 2025]라온시큐어 "인증 기술 토대 신사업 전개, 세계 무...
인젠트, 홈플러스 클라우드 데이터 이관 시스템 구축 완료
경기도교육정보기록원, 2025년 나이스 교무업무 학년초 사용자 연수 추진
[최기일의 방산보국㉘] "방위산업 분야 블록체인 플랫폼 구축해야" - 아시아에이
블록체인, 우라늄 시장 토큰화로 새로운 혁신 기대
허창언 보험개발원장 "빅데이터와 AI 활용 보험산업 플랫폼 강화할 것"
뉴스 없음
'채용 혹한기' 보낸 IT업계…유일하게 생존한 'AI 개발자'
[INTERVIEW 360] #AI시대 #클라우드...보안이 생명인 기업의 선택, ‘넷킬러(NETKILLER)’


In [109]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment

# 엑셀 파일 경로
excel_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"
new_excel_path = "SGA업계 및 경쟁사 현황4_업계별_현황.xlsx"  # 새 파일 경로
sheet_name = "업계현황"
output_sheet_name = "업계현황"  # 새로운 시트 이름

# 데이터프레임 로드 (bs_info_df, CIS_info_df는 미리 정의되어 있어야 함)
# 예시로 df를 bs_info_df로 지정합니다. 필요에 따라 변경하세요.


# 기존 엑셀 파일 불러오기 (서식 유지)
wb = load_workbook(excel_path)
ws_old = wb[sheet_name]

# 시트 이름이 기존에 존재하는지 확인
if output_sheet_name in wb.sheetnames:
    ws = wb[output_sheet_name]  # 기존 시트를 선택
    print(f"기존 시트 '{output_sheet_name}'에 데이터를 추가합니다.")
else:
    ws = wb.copy_worksheet(ws_old)  # 기존 시트를 복제
    ws.title = output_sheet_name  # 복제한 시트의 이름 변경
    print(f"새로운 시트 '{output_sheet_name}'을(를) 생성하였습니다.")

# 엑셀의 3행부터 14행까지 있는 회사명 찾기
target_rows = range(3, 17)  # openpyxl은 1-based index (pandas는 0-based index)
for row in target_rows:
    company_name = ws.cell(row=row, column=1).value  # 회사명 가져오기
    if company_name in df_top_news['topic'].values:
        # 'new_title' 값 가져오기
        title_value = df_top_news.loc[df_top_news['topic'] == company_name,'new_title'].values[0]
        if title_value == '뉴스 없음':
            print(f"'{company_name}'의 'new_title'이 '제목 없음'이므로 건너뜁니다.")
            continue  # 'new_title'이 '제목 없음'이면 해당 행 건너뛰기
        # 'new_title' 값을 2열에 추가
        title_cell = ws.cell(row=row, column=2, value=title_value)
        # 'org_link'를 하이퍼링크로 추가
        org_link_value = df_top_news.loc[df_top_news['topic'] == company_name, 'org_link'].values[0]
        title_cell.hyperlink = org_link_value
        title_cell.font = Font(color="0000FF", underline="single")
        # 'imp_sent' 값을 3열에 추가
        imp_sent_value = df_top_news.loc[df_top_news['topic'] == company_name, 'imp_sent'].values[0]
        sentence_cell = ws.cell(row=row, column=3, value=imp_sent_value)
        # 셀 서식 설정 (예: 글꼴 굵게, 가운데 정렬)

        title_cell.alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')
        sentence_cell.alignment = Alignment(wrap_text=True,horizontal='center', vertical='center')
    else:
        print(f"데이터프레임에 '{company_name}'에 대한 정보가 없습니다.")

# 엑셀 파일 저장
wb.save(new_excel_path)
print(f"데이터프레임이 '{new_excel_path}' 파일에 성공적으로 추가되었습니다.")


기존 시트 '업계현황'에 데이터를 추가합니다.
데이터프레임에 '제로트러스트'에 대한 정보가 없습니다.
'망분리'의 'new_title'이 '제목 없음'이므로 건너뜁니다.
데이터프레임에 '기타'에 대한 정보가 없습니다.
데이터프레임에 'None'에 대한 정보가 없습니다.
데이터프레임에 'None'에 대한 정보가 없습니다.
데이터프레임이 'SGA업계 및 경쟁사 현황4_업계별_현황.xlsx' 파일에 성공적으로 추가되었습니다.
